## Setup

In [2]:
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import MultiLabelBinarizer

https://curiousily.com/posts/multi-label-text-classification-with-bert-and-pytorch-lightning/
https://www.youtube.com/watch?v=vNKIg8rXK6w&ab_channel=rupertai


In [3]:
# Set paths
path = "."
os.chdir(path)
data_path = path + "/data"
output_path = path + "/outputs"
fig_path = path + "/figs"

In [4]:
# Suppression des FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

## Create Dataset

In [5]:
# Import data
df = pd.read_pickle(os.path.join(data_path, 'working_data_sans_dewey.pkl'))
print("Original dataset: ", df.shape)
train_df = pd.read_csv(os.path.join(data_path, 'train_dataset.csv'))
print("train dataset: ", train_df.shape)
test_df = pd.read_csv(os.path.join(data_path, 'test_dataset.csv'))
print("test dataset: ",test_df.shape)
valid_df = pd.read_csv(os.path.join(data_path, 'valid100_dataset.csv'))
print("validation dataset: ", valid_df.shape)

Original dataset:  (154508, 10)
train dataset:  (125264, 10)
test dataset:  (29244, 10)
validation dataset:  (100, 10)


In [6]:
# Transform rameau_list_unstack variable
rameau = pd.read_csv(os.path.join(data_path, "rameau_Tf_Td.csv"))
print("Taille du catalogue RAMEAU : ", rameau.shape)

Taille du catalogue RAMEAU :  (103628, 4)


In [7]:
# Convert the categorical labels to Multi Label Encodings
mlb = MultiLabelBinarizer()
mlb.fit([rameau["NOM"]])


MultiLabelBinarizer()

In [8]:
# Verification des classes
print(f"There are {len(mlb.classes_)} different Rameau PPN")
mlb.classes_[0:10]

There are 103021 different Rameau PPN


array(['!Xóõ (langue)',
       '"Sprach- und Sachatlas Italiens und der Südschweiz"',
       '"Taalatlas van Noord- en Zuid-Nederland"', "'?d", "'?ntokyo",
       "'Are'are (peuple des îles Salomon)", "'Au keto",
       "'Au keto, Musique d'", "'Au ni aau", "'Au ni aau, Musique d'"],
      dtype=object)

In [9]:
# Binarize rameau concepts in train/test and valid dataset
train_labels = mlb.transform(train_df["rameau_concepts"].apply(lambda x: eval(x)))
test_labels = mlb.transform(test_df["rameau_concepts"].apply(lambda x: eval(x)))
valid_labels = mlb.transform(valid_df["rameau_concepts"].apply(lambda x: eval(x)))

# Check binarization
print("train labels:", mlb.inverse_transform(train_labels[0:5]))
print("test labels:", mlb.inverse_transform(test_labels[0:5]))
print("valid labels:", mlb.inverse_transform(valid_labels[0:5]))

/home/aurelie/anaconda3/envs/abes_index/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(
/home/aurelie/anaconda3/envs/abes_index/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [''] will be ignored
  warnings.warn(


train labels: [('Culture populaire', 'Diffusion de la culture', 'Politique culturelle'), ('Complexe de castration', 'Psychanalyse', 'Rêves'), ('Culture',), ('Science-fiction américaine', 'Traductions françaises'), ('Photographie',)]
test labels: [('Structuralisme',), ('Mathématiques économiques', 'Théorie des jeux'), ('Crise économique (1973)',), ('Magie',), ('Linguistique', 'Linguistique mathématique', "Manuels d'enseignement", 'Mathématiques', 'Méthodes graphiques', 'Théorie des ensembles', 'Théorie des graphes', 'Étude et enseignement')]
valid labels: [('Bureaucratie', 'Classes dirigeantes', 'Hauts fonctionnaires', 'Hommes politiques', 'Pouvoir (sciences sociales)', 'Pratiques politiques'), ('Dollar américain', 'Finances internationales', 'Politique monétaire'), ('Intellectuels', 'Intellectuels français'), ('Bouddhisme',), ('Histoire', 'Mathématiques', "Psychologie de l'apprentissage", 'Sociologie', 'Étude et enseignement')]


In [10]:
# Create dataframe
df_train = pd.DataFrame(train_labels, columns=mlb.classes_)
df_test = pd.DataFrame(test_labels, columns=mlb.classes_)
df_valid100 = pd.DataFrame(valid_labels, columns=mlb.classes_)


In [11]:
# Add Description
df_train["descr"] = train_df["DESCR"]
print("train dataset :", df_train.shape)
df_test["descr"] = test_df["DESCR"]
print("test dataset :", df_test.shape)
df_valid100["descr"] = valid_df["DESCR"]
print("valid dataset :", df_valid100.shape)

train dataset : (125264, 103022)
test dataset : (29244, 103022)
valid dataset : (100, 103022)


In [12]:
# get one row
row_id = 64
sample_row = df_train.iloc[row_id]
sample_descr = sample_row.descr
sample_labels = sample_row[mlb.classes_]

print("Description: ", sample_descr)
print("Concepts: ", sample_labels[sample_labels != 0].to_dict())


Description:  La bataille mondiale des matières premières Dans le débat sur un nouvel ordre économique international, les marchés mondiaux des matières premières constituent un enjeu de première importance. Ils conditionnent largement les moyens de financement du développement de pays pauvres et sont un des lieux stratégiques où se joue l'indépendance des pays. L'auteur analyse d'abord les mécanismes et les acteurs des marchés libres, mettant en lumière les limites du jeu libéral de l'offre et de la demande. Son examen des divers systèmes de régulation qui ont été expérimentés l'amènent ensuite à émettre de sérieuses réserves sur l'efficacité des stocks régulateurs. De même, les accords compensatoires (type prêts du FMI) se heurtent-ils à des difficultés théoriques et concrètes de mise en place. La régulation de l'offre n'a véritablement réussi que dans le cas du pétrole. Des solutions plus radicales existent en dehors d'un fonctionnement aménagé du marché : ouverture unilatérale des f

In [13]:
## Save data
df_train.to_pickle(os.path.join(data_path, "train_dataset_for_DL.pkl"))
df_test.to_pickle(os.path.join(data_path, "test_dataset_for_DL.pkl"))
df_valid100.to_pickle(os.path.join(data_path, "valid100_dataset_for_DL.pkl"))

In [ ]:
df_train.to_csv(os.path.join(data_path, "train_dataset_for_DL.csv"))
df_test.to_csv(os.path.join(data_path, "test_dataset_for_DL.csv"))
df_valid100.to_csv(os.path.join(data_path, "valid100_dataset_for_DL.csv"))

In [17]:
# list of labels
pd.DataFrame(mlb.classes_, columns=["label"]).to_csv(os.path.join(data_path, "labels.txt"), index=0)